In [13]:
import os
import sys
import pymongo
import numpy as np
import pandas as pd
from decouple import config

## Табицы с полем «Эконометрика-2»

In [14]:
client = pymongo.MongoClient(config('DB_ADDRESS'))
cacs_api = client['cacs_api']

In [15]:
def add_econometrics_selecton(file_name):
    df = pd.read_excel('../saved/' + file_name, index_col='Unnamed: 0', engine='openpyxl')

    for i in df.index:
        st_hash = df.loc[i,'hash']
        user = cacs_api.users.find_one({'hash': st_hash})
        all_student_lessons = cacs_api.all_students_schedule.find({'stid': user['id']})
        has_econometrics = 0
        
        for lesson in all_student_lessons: 
            if lesson['subject'] == 'Эконометрика-2':
                has_econometrics = 1
                break
    
        df.loc[i, 'Эконометрика-2'] = has_econometrics
        
    df.to_excel('../saved/with_ekm/' + 'result_' + file_name)

In [17]:
if 'with_ekm' not in os.listdir('../saved/'):
    os.mkdir('../saved/with_ekm')

for i in os.listdir('../saved/'):
    if 'hash_' in i:
        print(i)
        add_econometrics_selecton(i)

hash_res_301.xlsx
hash_res_302.xlsx
hash_res_303.xlsx
hash_res_304.xlsx
hash_res_305.xlsx
hash_res_306.xlsx
hash_res_307.xlsx
hash_res_308.xlsx
hash_res_309.xlsx
hash_res_310.xlsx
hash_res_311.xlsx
hash_res_312.xlsx


## Сводная табица

In [19]:
def merge_vertically(names):
    
    dataframes_to_stack = []

    for file_name in names:
        
        df = pd.read_excel('../saved/with_ekm/' + file_name, index_col='Unnamed: 0', engine='openpyxl')
        dataframes_to_stack.append(df)
    
    df_result = pd.concat(dataframes_to_stack, axis=0)
    
    df_result.to_excel('../saved/with_ekm/result.xlsx')

In [20]:
merge_vertically([file_name for file_name in os.listdir('../saved/with_ekm') if file_name != 'result.xlsx'])

In [21]:
df = pd.read_excel('../saved/with_ekm/result.xlsx', index_col='Unnamed: 0', engine='openpyxl')

y = df['Эконометрика-2']

df1 = df[['hash', 'birth', 'contract', 'group']]
df2 = df.drop(['hash', 'birth', 'contract', 'group', 'Эконометрика-2'], axis = 1)

df_result = pd.concat([df1, df2, y], axis=1)
df_result = df_result.reset_index(drop = True)

df_result

,hash,birth,contract,group,Алгоритмические языки (VisualBasic) зачет л,Английский язык зачет 1,Английский язык зачет 3,Английский язык экзамен 2,Английский язык экзамен 4,Базовый инструментарий воспроизводимых научных исследований 2-Python зачет л,...,Фундаментальные представления современной физики: от взаимодействий элементарных частиц до структуры и эволюции Вселенной зачет 5,Международный бизнес зачет 5,Совершенствование природы человека: жизнь без старости зачет 5,Некоммерческие организации: правила игры зачет 5,Теория и практика ландшафтного дизайна зачет 5,Этика общения зачет 5,Бизнес-консалтинг; информатика и экономика зачет 5,Финансовое планирование бизнеса зачет 5,Введение в цифровую медицину зачет 5,Эконометрика-2
0,42ced8249b2949707fdf7e193d5612be,2001,без договора,301,3.0,3,3,5,5,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,2a84e771769d3e79eece8a6c9ef262be,2001,без договора,301,3.0,3,3,5,5,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,c78df235427408383a48ab758693ff53,2000,без договора,301,NaN,3,3,5,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,5a1d5f0cf4b151e007b63e50613be6c1,2000,без договора,301,NaN,3,3,5,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,070978ed9b0efaa20119c19b8146e1fa,1999,без договора,301,NaN,3,3,5,5,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,4527f80e3b488baa993ae854d7e6d0d7,2001,без договора,312,NaN,3,3,5,5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1
249,4b2ef6528c0faf8ffe6855758b89d0e6,2000,N 14-2-1-2018-0143 10...,312,NaN,3,3,5,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
250,0a1a595d8d26f46e30a8e8f376b71ed9,1999,без договора,312,NaN,3,3,5,5,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1
251,7428e2b479aeac3040c3fb2d130ade74,2000,без договора,312,3.0,3,3,5,5,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


## Количество пересдач

In [22]:
df_result.fillna(0, inplace = True)

df_result['Пересдачи'] = 0

for index, row in df_result.iloc[:, :-1].iterrows():
    if index > 4:
        counter = 0

        for i, s in enumerate(list(row)):
            if i > 4:
                cell_value = str(s).split(' ')

                if len(cell_value) > 1:
                    df_result.iloc[index, i] = cell_value[len(cell_value) - 1]
                    counter += len(cell_value) - 1

        df_result.loc[index,'Пересдачи'] = counter

df_result

,hash,birth,contract,group,Алгоритмические языки (VisualBasic) зачет л,Английский язык зачет 1,Английский язык зачет 3,Английский язык экзамен 2,Английский язык экзамен 4,Базовый инструментарий воспроизводимых научных исследований 2-Python зачет л,...,Международный бизнес зачет 5,Совершенствование природы человека: жизнь без старости зачет 5,Некоммерческие организации: правила игры зачет 5,Теория и практика ландшафтного дизайна зачет 5,Этика общения зачет 5,Бизнес-консалтинг; информатика и экономика зачет 5,Финансовое планирование бизнеса зачет 5,Введение в цифровую медицину зачет 5,Эконометрика-2,Пересдачи
0,42ced8249b2949707fdf7e193d5612be,2001,без договора,301,3.0,3,3,5,5,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
1,2a84e771769d3e79eece8a6c9ef262be,2001,без договора,301,3.0,3,3,5,5,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
2,c78df235427408383a48ab758693ff53,2000,без договора,301,0.0,3,3,5,5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,5a1d5f0cf4b151e007b63e50613be6c1,2000,без договора,301,0.0,3,3,5,5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
4,070978ed9b0efaa20119c19b8146e1fa,1999,без договора,301,0.0,3,3,5,5,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,4527f80e3b488baa993ae854d7e6d0d7,2001,без договора,312,0.0,3,3,5,5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1,0
249,4b2ef6528c0faf8ffe6855758b89d0e6,2000,N 14-2-1-2018-0143 10...,312,0.0,3,3,5,4,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,3
250,0a1a595d8d26f46e30a8e8f376b71ed9,1999,без договора,312,0.0,3,3,5,5,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1,2
251,7428e2b479aeac3040c3fb2d130ade74,2000,без договора,312,3.0,3,3,5,5,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


## Фиктивные переменные

In [23]:
zachet = [name for name in list(df_result) if 'зачет' in name.split(' ')]

for name in zachet:
    df_result[name] = df_result[name].apply(lambda x: x if x == 0 else 1)

df_result['contract'] = df_result['contract'].apply(lambda x: 0 if (x == 'без договора' or x == 0)  else 1)

for group in np.sort(df_result['group'].unique()):
    df_result[str(group)] = 0

for index, group in df_result['group'].items():
    df_result.loc[index, str(group)] = 1
    
df_result.drop(['group'], axis = 1)

df_result = pd.concat([df_result.iloc[:, :4], df_result.iloc[:, -12:], df_result.iloc[:, 4:-12]], axis = 1)

df_result = df_result.drop(['group', '301'], axis = 1)

In [24]:
df_result.iloc[:, 1:] = df_result.iloc[:, 1:].astype(int)

In [27]:
df_result.to_excel('../saved/with_ekm/result.xlsx')

In [84]:
groups = list(map(lambda x: (x['hash'], x['group']), list(cacs_api.users.find({'role': 'student'}))))
resulting_df = df_result[['birth', 'contract', 'hash', "Пересдачи","Эконометрика I экзамен 5",
                          "Линейная алгебра - l экзамен 1", "Математическая статистика экзамен 4",
                         "Теория вероятностей экзамен 3", "Эконометрика-2"]]
resulting_df['hash'] = resulting_df['hash'].apply(lambda x: [i[1][1:] for i in groups if x == i[0]][0]).copy()
print(resulting_df.columns)
resulting_df.columns = ['birth', 'contract', 'group', 'retest', 'ecm1', 'linal1', 'matstat', 'terver', 'ecm2']
resulting_df.to_excel('students_data.xlsx')

Index(['birth', 'contract', 'hash', 'Пересдачи', 'Эконометрика I экзамен 5',
       'Линейная алгебра - l экзамен 1', 'Математическая статистика экзамен 4',
       'Теория вероятностей экзамен 3', 'Эконометрика-2'],
      dtype='object')


<ipython-input-84-46b6daed206d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resulting_df['hash'] = resulting_df['hash'].apply(lambda x: [i[1][1:] for i in groups if x == i[0]][0]).copy()
